# 在现实世界中开发与部署 LLM 应用的最佳实践

---

你已经打造出一个功能强大的大语言模型（LLM）应用——它能回答复杂问题、生成优质文本，甚至能够推理棘手难题。真正的考验即将开始：把它从实验室带到真实用户面前。

在生产环境部署 LLM 不只是拥有一个优秀的模型，更重要的是构建一个快速、可靠、成本高效且可扩展的系统。

```
🚀 把这件事想象成发射航天器：模型是发动机，而架构则是制导系统、燃料箱和降落伞。没有扎实的计划，再先进的模型也可能坠毁。
```

在本章中，我们将带你走完部署大型语言模型的完整生命周期——从理解业务需求到优化性能、管理成本，再到确保系统稳定性。

## 回顾至此……

你已经准备好把 **TaskFriend** 推向更广泛的受众，并向公众开放。然而在此之前，你想确认自己面面俱到，确保系统在生产环境中依旧 **快速、可靠、成本高效且可扩展**——就像发射航天器一样，模型是发动机，架构则是制导系统、燃料箱和降落伞。

这意味着要验证从业务需求、模型选择到性能优化、用户体验与系统韧性等方方面面。

## 本章目标

* 学习如何把业务需求转化并平衡为清晰的技术要求。
* 理解从应用与用户角度优化 LLM 应用性能的技术手段。
* 掌握保障应用可靠性与韧性背后的核心考量。

## 初始化环境

### 配置 API 密钥

在开始使用任何笔记本之前，我们需要加载 [Model Studio 的 API 密钥](https://modelstudio.console.alibabacloud.com/?tab=globalset#/efm/api_key)。这样才能调用本课程中所用到的 Qwen 系列模型的 API。

> 如果你不确定如何获取 Model Studio API 密钥，请参考 `00-2 Setting Up Model Studio for LMP-C01.ipynb` 文件。

In [1]:
# Load Model Studio API key
import os
from config.load_key import load_key
load_key(
    confirmation=False
)

Detected existing API key: sk-8...2716
Auto-confirmation enabled. Using existing API key.


In [2]:
from openai import OpenAI
from functions.llm_utils import (
    get_qwen_stream_response,
    get_hardcoded_response,
    get_precomputed_response,
    benchmark_responses
)
import time

client = OpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1/",
)

# 将业务需求转化为技术规范

---

在建房子之前，你不会直接拿起锤子开始钉木板，而是先画蓝图——明确要建什么、服务于谁、应该如何运作。

部署大语言模型（LLM）也一样。在上线之前，你必须问自己：我们为什么要做这件事？要解决什么业务问题？

这听起来显而易见，但在 AI 项目中跳过这一步是最常见的失败原因之一。旅程从这里开始：弄清模型需要完成哪些任务、用户是谁、成功的衡量标准是什么。如果忽视这一步，最终得到的系统要么太慢、要么太贵、要么根本不可用。

想象你在构建客服机器人。如果没有搞清楚它是否需要实时应答、是否要支持多语言、是否要与 CRM 系统集成，你就可能选择错误的模型、付出过高的成本，或提供糟糕的用户体验。

让我们看看正确的做法。要做到这一点，先回答以下问题：

* 模型要执行哪些任务？
* 用户是谁？
* 性能期望是什么（例如响应时间、准确率）？
* 成本约束有哪些？

只有在这些问题都有明确答案时，你才能带着信心继续向前。

## 功能性需求：模型需要做什么？

功能性需求定义了模型应该完成的任务——它需要解决的问题。

这些就是用户所期望的核心能力，可以把它们看作模型的“功能”。

### 常见用例与模型选择
下面列出了一些典型的功能性需求及其最适合的模型类型：

这里有一张便捷的对照表：


| 用例                       | 推荐的模型类型                 | 示例                             |
|----------------------------|--------------------------------|----------------------------------|
| 通用问答                   | 通用型 LLM                     | Qwen、GPT、DeepSeek             |
| 数学问题求解               | 领域专用模型                   | Qwen-Math                       |
| 医疗诊断                   | 医疗领域 LLM + 知识图谱        | 专用医疗模型                    |
| 图像理解                   | 视觉模型                       | Qwen-VL                         |
| 图像与视频生成             | 扩散模型                       | WAN、Stable Diffusion           |
| 语音助手                   | 语音处理模型                   | CosyVoice、Qwen-Audio           |
| 多模态任务                 | 统一的多模态模型               | Qwen-VL、BLIP-2                 |

在确定需求之后，你会面临另一个挑战：市面上有大量性能相近、功能类似的模型（如 `Qwen`、`GPT`、`Claude`、`Gemini`、`DeepSeek` 等）。此时，需要基于具体用例进行评估——哪一个模型能在你的约束条件下表现最佳，并与业务需求最匹配？你可以用自有数据集构建评测，也可以使用公开数据集；此外还有大量基准可用于检验模型的实际表现。

<div style="text-align: center;">
  <img src="images/LMP-C01_10-LLMArena Leaderboard.png" style="max-width: 800px;" />
  <br>
  <small>LLMArena 排行榜</small>
  <br>
  <small><i>来源：<a href="https://lmarena.ai/leaderboard" target="_blank">LLMArena</a></i></small>
</div>


 > 🧠 **专业提示：**  
 > 别只盯着“最大的”模型。针对特定领域的小型或微调模型往往能胜过通用大模型——而且更快、更省钱。

### 如何选择合适的模型
当你确定了需求后，会紧接着面对另一个问题：

```
面对数十种模型可选，该如何挑到最合适的那一个？
```

从以下问题入手：

* 这是哪类任务？（例如：摘要、推理、代码生成）
* 是否存在可用的领域专用模型？
* 我需要极高准确率，还是速度更重要？



然后进行测试。

使用真实场景的数据评估模型。你可以构建自己的评测集，或使用以下公共基准：

* MMLU —— 通用语言理解
* BBH —— 复杂推理
* HumanEval —— 代码生成
* LongBench —— 长上下文任务


 > 🚀 **产品思维提醒：**  
 > 目标不是选择最准确的模型，而是选择最契合用例、用户期望与业务约束的模型。


## 非功能性需求：应用完成任务的质量标准是什么？

当你明确模型需要做什么之后，下一步就是定义它应该做到什么程度。

这就是非功能性需求的范畴。它们是那些看不见却至关重要的特性，决定了用户感知中的速度、可靠性与成本效益。

### 核心非功能性需求

| 因素        | 为什么重要                                                                      |
|-------------|-----------------------------------------------------------------------------------|
| 性能        | 用户期望快速响应——尤其在聊天机器人与实时系统中。                               |
| 成本        | 高昂的算力花费会迅速侵蚀利润，甚至让产品难以为继。                             |
| 稳定性      | 如果模型宕机或表现异常，用户会立刻失去信任。                                   |
| 安全性      | 敏感数据必须得到保护，特别是在金融、医疗等受监管行业。                         |
| 可扩展性    | 系统能否在流量突增时保持稳定，而不是直接崩溃？                                 |


### 性能：速度就是一切

假设你正在打造一个客服机器人。

如果回复需要 5-10 秒，用户会感到沮丧；如果它几乎瞬间回应，用户会觉得神奇。

这就是性能的力量。实现这一点的最佳方式是定义性能基线，专业术语称为服务级目标（SLO）。

下表列出了不同用例的推荐 SLO：

| 用例            | TTFT（首个 Token 时间） | TPOT（每个输出 Token 时间） |
|-----------------|-------------------------|------------------------------|
| 聊天机器人      | < 500ms                 | < 200ms                      |
| 代码补全        | < 300ms                 | < 100ms                      |
| 文本摘要        | < 1s                    | < 300ms                      |
| 批处理任务      | 视用例而定             | 视用例而定                  |


 > ⚠️ **警告：**  
 > 不要凭感觉估计这些数字。请结合真实用户进行测量。你觉得快的速度，用户可能仍然觉得慢。


### 成本：聪明部署，而非一味堆料

你或许很想直接使用最强大的模型——但那可能是昂贵的错误。

大型模型的运行成本极高。每一次推理调用都要花钱，当你需要为成千上万的用户服务时，费用会迅速叠加。

因此要进行优化。

能用小模型就别用大模型。缓存常见结果。批量处理请求。控制 Token 数量。

在使用云服务时，也要谨慎选择计费模式：

| 计费模式             | 最适合的场景                      | 优势                           | 劣势                       |
|----------------------|-----------------------------------|--------------------------------|----------------------------|
| 预付费（年/包年包月） | 稳定、可预测的工作负载            | 长期看来最便宜                | 需要预付成本               |
| 按量付费              | 用量不可预测                     | 无需预付费用                  | 单次成本更高              |
| 竞价实例              | 对成本敏感且可容忍故障的应用     | 最多可便宜 90%               | 有被中断的风险             |


 > 💡 **专业洞察：**  
 > 将策略组合使用。把后台任务放在竞价实例上运行，把实时请求留给按量付费。

### 稳定性：打造永不休眠的系统

即便模型表现再出色，如果每隔几个小时就崩溃一次，用户也不会买账。

稳定性意味着系统能处理真实世界的流量、在故障后恢复，并在高压下持续发挥。

问问自己：

* 你的模型能否在流量高峰时扩容？
* 如果模型失败，你有没有兜底方案？
* 你是否在监控性能并提前发现问题？

 > 🧱 **工程师 mindset：**  
 > 目标不是只构建一个模型，而是构建能够 7x24 稳定运行该模型的系统。

# 性能优化：让你的 LLM 反应更灵敏

---

你已经选好了合适的模型，也定义了性能目标。现在是时候让模型在压力下依旧快速、灵敏、高效。

性能不仅仅是速度，它还关乎用户体验、成本控制与系统可扩展性。缓慢的模型会让用户沮丧、烧光预算，还限制可服务的用户量。

本节我们将探讨优化 LLM 性能的关键技术：从模型压缩、缓存到并行处理与智能的 Token 管理。每种策略都源于真实项目与工程原则。

让我们开始吧。

## 系统性能提升的核心准则

把你的 LLM 想象成一辆高性能赛车的发动机。它很强大，但如果没有合适的变速箱、悬挂和空气动力学设计，也赢不了比赛。

系统性能优化的目标是调优整条技术栈，而不仅仅是模型；要降低延迟、提升吞吐量、减少资源消耗。

以下是五条需要遵循的核心原则：

| 原则 | 说明 |
|--------|-------------|
| ✅ **更快（且更高效）的请求处理** | 通过模型优化、量化和硬件加速缩短推理时间。 |
| 🔁 **减少请求次数** | 利用缓存、批处理与去重来避免重复计算。 |
| 📉 **减少输入与输出 Token 数量** | 精简不必要的上下文，只保留必要输出。 |
| ⚙️ **利用并行处理** | 在多设备或多阶段分摊工作，提升吞吐量。 |
| 🧱 **在更简单的方案可行时避免动用模型** | 适当使用硬编码、预计算或传统算法。 |

 > 💡 **黄金法则：**  
 > *最快的计算就是无需执行的计算。*

## 更快的请求处理：让模型思考得更快（也更高效）

影响推理速度的首要因素是模型规模。小模型就是更快。

你并不总是需要一个 6000 亿参数的巨兽才能获得优秀结果。在许多情况下，经过良好优化的 7B 或 14B 模型就能匹敌甚至超越大模型——尤其是在特定领域任务上。额外的好处是小模型占用资源更少，直接转化为成本节约。

那么小模型是如何做到的？简单来说，它们通常由大型模型通过各种技术“压缩”而来，方法包括：

```mermaid
graph LR
    subgraph flowchart[模型优化技术]
        LargeModel[大型教师模型] -->|知识蒸馏| SmallModel[小型学生模型]
        LargeModel -->|剪枝| PrunedModel[剪枝模型]
        PrunedModel -->|量化| QuantizedModel[INT4/INT8 模型]
    end
```

<table>
  <tr>
      <td><strong>技术</strong></td>
      <td><strong>工作原理</strong></td>
  </tr>
  <tr>
    <td><strong>模型剪枝</strong> <br><img src="images/LMP-C01_10-Model Pruning.png" alt="Model Pruning" style="max-width: 300px"><br><small><i>来源：<a href="https://arxiv.org/abs/1506.02626">Learning both Weights and Connections for Efficient Neural Networks</a></i></small></td>
    <td>移除模型中的冗余权重或层，降低模型复杂度。<br>就像修剪树木，让它长得更快。</td>
  </tr>
  <tr>
    <td><strong>量化</strong> <br><img src="images/LMP-C01_10-Quantization.gif" alt="Quantization" style="max-width: 300px"><br><small><i>来源：<a href="https://www.qualcomm.com/news/onq/2019/03/heres-why-quantization-matters-ai">Quallcomm</a></i></small></td>
    <td>将 FP32 权重量化为更低精度（如 INT8、INT4），减少内存与计算需求。</td>
  </tr>
  <tr>
    <td><strong>知识蒸馏</strong> <br><img src="images/LMP-C01_10-Knowledge Distillation.png" alt="Knowledge distillation" style="max-width: 300px"><br><small><i>来源：<a href="https://arxiv.org/abs/2006.05525">Knowledge Distillation: A Survey</a></i></small></td>
    <td>让小型“学生”模型学习大型“教师”模型的输出行为。<br><strong>示例：</strong><ul><li>DeepSeek-R1-Distill-Qwen-7B</li><li>DeepSeek-R1-Distill-Llama-8B</li></ul></td>
  </tr>
</table>

虽然模型压缩能带来巨大收益，但也要关注潜在权衡：

* **准确率损失：** 过度剪枝或激进量化会降低输出质量。
* **领域限制：** 蒸馏模型在领域外输入上可能表现不佳。
* **硬件兼容性：** 某些量化模型需要特定 GPU（例如 INT4 需要 NVIDIA A100）。
* **再训练成本：** 压缩通常需要重新训练或微调。

### 加分技巧：优化你的提示词

别不相信，提示词也会影响性能。一个设计良好的提示词能减少模型需要处理的 Token 数量，从而提升推理速度。

试试这些招数：

 > * **提示压缩：** 删除多余指令。
 > * **少样本优化：** 使用简洁示例。
 > * **提示词微调：** 学习一个精简前缀，帮助模型更高效地响应。


```mermaid
graph LR
    Prompt[原始提示词] --> Compressed[压缩后的提示词]
    style Prompt fill:#f9f,stroke:#333
    style Compressed fill:#bbf,stroke:#333

    subgraph Before
        Prompt -->|200 个 token| Model[LLM 推理]
    end

    subgraph After
        Compressed -->|70 个 token| Model
    end

    Model --> Response[响应]
```

## 减少请求次数（与计算负载）

每一次推理调用都会消耗时间与金钱。那么为何要让模型做更多的无用功呢？

聪明的系统会通过重用结果、合并请求、避免不必要的调用来尽量减少重复计算。

### 结合上下文复用的缓存
在聊天机器人或文档分析等场景中，用户常常提供重叠的上下文。

与其每次都重新处理相同信息，不如缓存共享上下文——例如对话历史或文档向量。

```mermaid
sequenceDiagram
    User->>Server: "基于我们的对话，我上一单买了什么？"
    Server->>Cache: 检查缓存上下文
    alt 命中缓存
        Cache-->>Server: 返回缓存的向量
        Server->>LLM: 使用缓存上下文进行推理
    else 未命中缓存
        Server->>LLM: 处理完整上下文
        Server->>Cache: 存储新的上下文
    end
    Server->>User: 返回结果
```

**影响：**  
上下文缓存可以把推理时间缩短最多 50%，并显著降低 GPU 占用。

 > 🧪 **专业提示：**  
 > Qwen 系列模型原生支持上下文缓存。确保调用 API 时使用正确参数以启用该能力。

### 批处理：事半功倍

批处理就像一次把衣服都洗了，而不是一只袜子一只袜子洗。把相似请求聚在一起处理，可以减少开销、提升硬件利用率。

这在以下场景尤为有用：

* 离线摘要
* 数据分类
* 报告生成
* 数据标注流水线

```mermaid
graph LR
    subgraph 未使用批处理
        R1[请求 1] --> GPU
        R2[请求 2] --> GPU
        R3[请求 3] --> GPU
        GPU --> O1[输出 1]
        GPU --> O2[输出 2]
        GPU --> O3[输出 3]
    end

    subgraph 使用批处理
        Batch[批次：R1+R2+R3] --> GPU
        GPU --> BatchOut[输出：O1+O2+O3]
    end
```

**影响：**  
批处理可以让吞吐量提升 3 倍以上，并显著降低 GPU 成本。

 > **阿里云小贴士：**  
 > **Model Studio** 提供 `Batch API`，可在离峰时段执行推理，从而节省 GPU 成本。

## 减少输入与输出 Token：少即是多

模型需要处理的 Token 越多，速度越慢——成本也越高。

每个 Token 都会增加计算时间，尤其是在自回归解码（输出生成）阶段。

那为何不减少 Token 数量呢？

### 输入优化：删繁就简

别把所有东西都丢给模型。只提取它真正需要的信息。

**策略包括：**

* **意图识别：** 使用轻量模型（如基于 BERT 的分类器）提取核心意图，再交给 LLM。
* **摘要预处理：** 面对长文档先做摘要。
* **基于规则的过滤：** 去掉页眉、页脚等模板内容。

**专业提示：**  
如果要处理超长上下文，可以考虑 **分层摘要**。它通过将长文档拆成更小片段逐步摘要，既能减少 Token，又能保留关键信息，让推理更快。

```mermaid
flowchart LR
    subgraph flowchart[分层摘要]
    direction LR
        LongDoc["长文档<br>(5,000 tokens)"] --> Summarizer1[分段摘要]
        subgraph Summary 1
            Summarizer1 --> MidSummary["中间摘要<br>(800 tokens)"]
        end
        MidSummary --> Summarizer2[最终摘要]
        subgraph Summary 2
            Summarizer2 --> ShortSummary["精简摘要<br>(200 tokens)"]
        end
        ShortSummary --> LLM[LLM 推理]
    end
```

### 输出优化：言简意赅

生成输出常常是推理流程中最耗时的部分。

要聪明地控制它：

* 使用简短提示，例如 `“请用一句话回答”` 或 `“列出最重要的 3 点”`。
* 通过 `max_tokens` 参数限制输出长度。
* 避免 `Explain everything` 之类的开放式请求。

**真实案例：**  
一个客服机器人每条回复生成 `200 个 token` 可能需要 2 秒；把输出缩短到 `50 个 token`，时间和成本都会明显下降。


## 并行处理：分而治之

LLM 计算量巨大，但这并不意味着它必须很慢。

通过并行化，你可以将工作拆分到多块 GPU 或多台机器上，大幅缩短推理时间。

主要有三种并行方式：

| 类型              | 工作原理                                   | 最适用的场景                              |
|-------------------|--------------------------------------------|-------------------------------------------|
| 数据并行          | 将输入数据拆分到不同设备                  | 批量推理                                   |
| 模型并行          | 将模型拆分到不同设备                      | 单卡放不下的大模型                         |
| 流水线并行        | 将推理流程拆分成多个阶段                  | 长序列或多阶段任务                         |

 > 🚀 **示例：**  
 > 如果你运行一个 100B 模型，可能需要模型并行把层分布到多块 GPU 上；但若要处理 1000 条短请求，数据并行更能带来速度提升。



## 避免默认依赖模型：有时简单更好

人们很容易陷入这样的思维：

```
“如果 70B 模型表现不错，那 600B 一定更好。”
```

但事实并非如此——成本更是如此。

有时候，最佳性能来自“不使用 LLM”。

下面是一些聪明的替代方案：

### 用预定义逻辑替代推理

当响应是可预测、重复或基于有限输入集合时，可以考虑用预定义逻辑取代实时 LLM 调用。包括：

* **硬编码** 高频固定回复
* **预计算** 已知输入模式的响应

两者结合形成一个强大的模式：`“先响应，再按需计算”`。

#### 硬编码 vs. 预计算：如何抉择

| 特性 | **硬编码** | **预计算** |
|--------|----------------|--------------------|
| **定义** | 在代码中手写固定响应。 | 为已知输入提前生成并存储响应。 |
| **最适用场景** | 静态、高频响应：<br>• `"Invalid input"`<br>• `"Request submitted"`<br>• `"Hello! How can I help?"` | 输入空间有限：<br>• 产品 FAQ<br>• 表单校验规则<br>• API 状态码<br>• 工单分类 |
| **示例** | ```python<br>if query == "help":<br>    return "Available commands: status, help, support"<br>``` | ```json<br>{<br>  "refund policy": "Returns within 30 days eligible for refund.",<br>  "shipping": "Free shipping on orders over $50."<br>}<br>``` |
| **延迟** | ⚡ 近乎为零（~0.1 ms） | 🚀 极低（使用缓存/数据库查询约 1–10 ms） |
| **单次成本** | 💵 基本为 \$0 | 💵 接近 \$0 |
| **维护成本** | 更新需要改代码 | 可通过配置、数据库或 CMS 更新 |
| **可扩展性** | 低——不适合上百条回复 | 高——适合结构化数据 |
| **适用情境** | ✅ 响应完全不变<br>✅ 频率极高<br>✅ 对速度要求极致 | ✅ 输入空间有限<br>✅ 响应可预测<br>✅ 需要避免实时推理 |
| **权衡** | ❌ 难以在大规模下维护<br>❌ 更新需重新部署 | ❌ 需要前期搭建<br>❌ 需要存储/缓存层 |

**快速决策指南**

| 你应该选择… | 如果… |
|------------------|-----|
| **硬编码** | 你在回复 `status`、`help`、问候语等常见命令，而且答案永远相同。 |
| **预计算** | 你有目录、FAQ 或规则集，输入可枚举且输出一致，想以极低延迟返回结果。 |

 > 💡 **专业提示：**   
 > 两者结合效果更佳！用 **硬编码** 处理系统核心消息，**预计算** 承载领域知识（如产品详情、政策）。

让我们快速试试看：

In [3]:
# Define cases
# Hardcoded responses to queries
HARDCODED_RESPONSES = {
    "status": "Your request has been successfully submitted and is currently being processed.",
    "help": "Available commands: status, help, support, billing.",
    "greeting": "Hello! How can I assist you today?",
}

# Hardcoded responses for queries
PRECALCULATED_KNOWLEDGE_BASE = {
    "how to reset password": (
        "To reset your password: 1. Go to Settings > Account. "
        "2. Click 'Reset Password'. 3. Check your email for a verification link."
    ),
    "refund policy": (
        "Our refund policy allows returns within 30 days of purchase. "
        "Visit the Billing section to initiate a refund request."
    ),
    "api documentation": (
        "Full API docs are available at https://api.yourservice.com/docs. "
        "Includes examples, auth guide, and rate limits."
    ),
}

# Queries for all 3 types of responses
QUERIES = {
    "hardcoded": "status",
    "precomputed": "how to reset password",
    "llm": "Explain quantum entanglement in simple terms"
}

In [ ]:
# Run benchmark for 3 types of responses
benchmark_responses(
    hardcoded_responses=HARDCODED_RESPONSES,
    precomputed_knowledge_base=PRECALCULATED_KNOWLEDGE_BASE,
    queries=QUERIES
)

### 传统 UI 组件：用可视化呈现信息

有时展示信息最好的方式并不是长篇大论，而是使用 **表格、图表或进度条** 等结构化 UI 组件。

它们具有以下优势：
- **渲染更快**（无需生成长文本）
- **更易于浏览理解**
- **比自然语言生成更高效**

与其让 LLM 用段落描述数据，不如让它输出结构化结果，再用 Markdown 表格等形式呈现。

**示例：**

把这段 LLM 文本输出……

 > “用户提交了三笔订单。订单 #1001 于周一下单，目前待处理。订单 #1002 于周二发货。订单 #1003 在周三送达。目前没有正在处理的订单。”

换成这张 Markdown 表格

| 订单号 | 日期       | 状态       |
|--------|------------|------------|
| #1001  | 周一       | 待处理     |
| #1002  | 周二       | 已发货     |
| #1003  | 周三       | 已送达     |

 > 💡 **为何更优：**  
 > * 用户能立刻看到状态与时间线  
 > * 无需逐句解析  
 > * 便于排序、过滤或扩展  
 > * 可直接由类似 JSON 的 LLM 输出转换而来

即便借助 LLM，也请提示它输出结构化数据（如 JSON、键值对），再将其渲染为表格，而不是段落。

 > 🎯 **专业提示：**  
 > 如果响应包含列表、对比或状态——**优先考虑用表格而不是句子**。

# 用户体验优化：不止于速度

---

速度很重要——但它并不是影响用户体验的唯一因素。  
即使系统很快，如果用户得不到反馈、错误处理糟糕或缺乏交互指引，他们仍然会觉得慢、坏、难用。

真正卓越的用户体验来自于 **感知、清晰度与信任** 的设计，而不只是低延迟。

本节我们将探讨实用技巧，帮助你的 LLM 应用在后端仍在工作时，也能让用户感到 **响应迅速、可靠、以用户为中心**。

这些策略聚焦于系统在交互过程中 **如何与用户沟通**——打造顺畅、可预期且令人安心的体验。

## 基础 UX 增强手段

### 流式输出：制造实时响应的错觉

不要让用户等到整段回答生成完毕才看到内容。  
使用 **流式模式** 在生成 Token 的同时展示结果，让用户感到系统正在与他们“共同思考”，而不是“单向输出”。

<div style="text-align: center;">
  <img src="images/LMP-C01_10-Qwen Stream Response.gif" style="max-width: 800px;" />
  <br>
  <small>Qwen 流式模式演示</small>
  <br>
  <small><i>来源：<a href="https://chat.qwen.ai" target="_blank">Qwen</a></i></small>
</div>


**好处包括**：
- 降低感知延迟
- 保持用户参与感
- 对话更像真实交流

 > **专业提示：**  
 > 在首个 Token 到达前显示一个简洁的“正在输入”指示（如 `💬`）。


In [4]:
# Query & parameters
query = "Explain why the sky is blue."
system_prompt = """
        You are an educator for 5-year olds. \
        When explaining concepts, make them easy to understand and read. \
        Limit yourself to 5 sentences.
    """
temperature = 0.5
top_p = 0.9

# Streaming response
response, elapsed = get_qwen_stream_response(
    query=query,
    system_prompt=system_prompt,
    temperature=temperature,
    top_p=top_p
)


📡 Calling LLM (Qwen)... 
The sky looks blue because of sunlight and tiny air particles.  
Sunlight looks white, but it has all the colors of the rainbow.  
Blue light bounces off the air particles more than other colors.  
It scatters all around the sky, like glitter in a jar.  
So when we look up, we see blue everywhere!
⏱️  Total time: 2.41s



### 展示任务进度：让用户知情

对于多步骤任务（如文档分析、报告生成），**将流程拆分成阶段** 并展示进度。

不要只是显示：
> “正在处理你的请求……”

可以换成：
> 📝 正在分析文档…  
> 🔍 正在提取要点…  
> 📄 正在生成摘要…

或者使用简单的 Markdown 进度条：

```markdown
[██████░░░░] 60% 完成 – 正在生成摘要
```

为何有效：

* 设定清晰预期
* 在长时间等待时降低焦虑
* 让系统看起来更透明

 > **专业提示：**  
 > 如果可以，给进度指示配上轻量动画，让用户感受到系统“活着”。


In [5]:
from IPython.display import clear_output

# Create function to show progress indicator (simulated)
def analyze_document_with_progress(doc, question):
    """
    Analyze a document with simulated progress and final LLM call.
    """
    steps = [
        "📝 Parsing document structure...",
        "🔍 Extracting key entities (products, dates, policies)...",
        "🧠 Determining response type (summary, comparison, FAQ)...",
        "📦 Preparing final prompt for LLM..."
    ]

    # Simulate progress
    for i, step in enumerate(steps):
        print(step)
        time.sleep(0.8)
        clear_output(wait=True)
        progress = "█" * (i + 1) + "░" * (len(steps) - i - 1)
        print(f"Progress: {progress} {int((i+1)/len(steps)*100)}%")
        print(f"Step {i+1}/{len(steps)}: {step}\n")

    print("✅ Starting LLM generation...\n")

    # Final LLM call
    full_prompt = f"""
    Based on the following document, answer the question concisely.
    If the answer isn't present, say "Not found in document."

    Document:
    {doc[:4000]}

    Question:
    {question}
    """

    return get_qwen_stream_response(
        query=full_prompt,
        system_prompt="You are a document analyst. Be accurate and concise.",
        temperature=0.3,
        top_p=0.9
    )

In [6]:
document = """
    NovaTech Solutions has been a leader in cloud infrastructure since 2015.
    We serve over 10,000 customers globally with data centers in North America, Europe, and Asia.

    Key Offerings:
    - Virtual Machines (NovaCloud VM): Scalable compute instances with GPU support.
    - Object Storage (NovaStore): Secure, durable storage with 99.99% uptime SLA.
    - AI Training Platform (NovaTrain): End-to-end environment for training LLMs.
    - Monitoring Tools (NovaWatch): Real-time dashboards and alerting.

    Recent Updates:
    - Q2 2024: Launched NovaTrain Pro with distributed training support.
    - July 1: All new accounts now include DDoS protection by default.
    - API rate limits are now tiered:
      • Free Tier: 100 requests/minute
      • Pro Tier: 1,000 requests/minute
      • Enterprise: Unlimited with priority routing

    Support: Available via email (support@novatech.com) and live chat (9 AM – 6 PM EST).
"""

In [7]:
# Analyze & query the document, showing the progress
question = "What are the key offerings of NovaTech?"
response, t = analyze_document_with_progress(document, question)

Progress: ████ 100%
Step 4/4: 📦 Preparing final prompt for LLM...

✅ Starting LLM generation...

📡 Calling LLM (Qwen)... 
The key offerings of NovaTech are:
- Virtual Machines (NovaCloud VM)
- Object Storage (NovaStore)
- AI Training Platform (NovaTrain)
- Monitoring Tools (NovaWatch)
⏱️  Total time: 1.06s



### 健壮的错误处理：优雅应对突发状况

错误总会发生——但你如何呈现，决定了用户是否信任你。

避免类似这样的原始报错：

> `Error: Model inference failed (status 500)` 

请改用友好且可采取行动的提示：


 > `❌ 当前请求暂时无法处理。`  
 > `请检查输入后重试。`  
 > `若问题持续，请联系支持团队。` 

技术细节请留在服务端日志中。

**关键原则：**

* 不要暴露内部系统错误
* 提供明确的下一步
* 保持品牌调性与语气

 > **专业提示：**  
 > 在展示错误前，自动对短暂故障（例如超时）发起重试。 


## 其他 UX 增强手段

### 引入反馈机制：让用户感觉被倾听

通过让用户直接评价响应，你可以让他们表达观点、提出建议（这在面向细分领域的 LLM 中尤为重要）。常见的反馈机制如下：

 > `这个回答有帮助吗？`  
 > `👍 有 👎 没有`

这些数据可以用于：

* 识别流程中的薄弱环节
* 重新训练或微调模型
* 将低置信度请求转交人工审核

### 智能跟进：预测用户的下一步

在交付答案之后，不要让用户不知所措。优秀的 LLM 应用会预判下一步，并引导对话继续。

**为何重要：**

* 前瞻性建议能降低摩擦、提升互动
* 它模拟自然对话——一个问题往往引出下一个

# 可靠性与韧性：打造不轻易出问题的系统

---

你已经构建了聪明的应用——现在要确保它能扛得住真实世界的流量、故障与突发峰值。在生产环境中，可靠性与准确性同样重要。

我们可以把应用的可靠性与韧性拆解成两个支柱：**基础设施管理** 与 **请求处理**。

## 基础设施：稳定性的地基
### 基线管理

建立性能基线是任何 LLM 应用生命周期中的关键第一步。团队不应直接跳到复杂模型，而应从简单、易解释的方法（如基于规则的系统或轻量算法）开始，为后续模型设定最低性能门槛。这个基线是评估未来模型的参考点，确保每次升级都确实带来提升。通过定期将新模型与基线进行对比、覆盖真实场景，并把自动化检查集成到部署流水线中，组织就能保持模型可靠性、适应环境变化，并通过金丝雀发布等策略安全地推新。

其他基线管理方法包括：

* 模型版本管理：固定模型版本，避免自动升级。
* 环境一致性：确保开发、预发与生产环境尽可能一致。
* 配置即代码：把配置（提示词、参数、API 密钥）存放在受版本控制的加密密钥中。

 > **专业提示：**  
 > 使用特性开关（Feature Flag）在不同模型版本或提示词之间切换，无需重新部署。

### 自动扩缩容

LLM 工作负载常常呈现突发性——用户提问并不会均匀分布。聪明的扩容策略包括：

* 水平扩展：在高峰期增加推理实例。
* 无服务器选项：使用如阿里云 Model Studio，可在空闲时缩容至零。
* 排队系统：对于耗时任务，使用消息队列（如 RabbitMQ、Kafka）将请求与处理解耦。

 > **示例：**  
 > 入职周期间，你的 HR 机器人流量暴涨 10 倍。自动扩缩容可以保证响应依旧迅速，同时在空闲期不额外付费。

### 灾备恢复

假设故障总会发生，并提前做好计划。

* 兜底模型：当主力 LLM（如 Qwen-72B）不可用时，切换至更小更快的模型（如 Qwen-1.8B）。
* 断路器：如果错误率超限（如 50% 超时），暂时停止调用。
* 数据备份：定期备份 RAG 索引、日志与用户反馈。

 > **恢复目标：**  
 > `“即便 LLM 宕机，系统也应该优雅降级，而非整体崩溃。”`

## 请求处理：掌控请求流量

### 限流

防止滥用，保障公平使用。

* **按用户限额：** 例如免费用户每小时 100 次，付费用户 1,000 次。
* **基于 Token 的节流：** 既限制请求次数，也限制输入+输出 Token。
* **动态限额：** 根据系统负载或用户角色调整。

🛑 为什么？没有限流，单个用户就能耗尽 LLM 配额，让其他人变慢。

### 监控与错误处理

你无法改进自己不监测的东西。以下是一些可长期跟踪的指标：

| 指标    | 作用    |
|-----------|----------------------|
| 延迟     | 推理是否在变慢？    |
| 错误率   | 是否出现超时或 5xx 错误？    |
| Token 使用 | 提示词是否过长？    |
| 缓存命中率 | 是否在有效复用响应？    |


**错误处理最佳实践：**
* **返回优雅的兜底内容：** `“我暂时无法直接回答，但这里有帮助页面链接。”`
* **完整记录上下文：** 输入、模型版本、响应时间与错误信息。
* **异常告警：** 如幻觉率或延迟突然上升。

 > **专业提示：**  
 > 把 LLM 当成一个微服务来管理——因为事实确实如此。套用 SRE 原则：SLA、SLO 与事后复盘。

# 成本考量：打造财务上可持续的应用

---

## 性能优化也能降低成本

前面提到的许多性能优化不仅减少延迟，还能显著降低运维开销。快速回顾：

* **能用小模型就用小模型：** 小模型推理更快、运行成本更低，非常适合不需要大型模型全部能力的任务。
* **上下文缓存：** 对重复问题缓存结果，免去多次调用 LLM。在阿里云上，Qwen 模型的 `cache_token` 成本仅为 `input_token` 的 40%，因此缓存非常划算。
* **批量推理：** 合并或去重请求，减少不必要的模型调用。批处理通常适合非实时任务，可安排在离峰时段利用闲置算力进一步节省成本。例如在阿里云百炼平台，批量推理的价格只有实时推理的 50%。
* **减少 Token 使用：** 精简输入、限制输出 Token，可直接降低计算负载与硬件/能源成本。
* **避免过度依赖大模型：** 把简单或确定性任务（如规则逻辑、数据格式化）交给传统代码或预计算系统，避免昂贵的 LLM 推理。

这些策略具有普适性。在真实部署中，主要存在两种模式：自建机房与云部署。自建方案前期投入高、采购周期长、运维复杂；云部署更适合初创团队与成本敏感应用。云平台能帮你托管基础设施、按需扩缩，确保资源利用高效。

### 云上成本优化

LLM 往往涉及大规模数据存储、高性能计算和复杂推理工作负载，因此可能带来高昂开销。本节探讨在云端部署 LLM 应用时如何做出高性价比的架构选择。

#### 选择合适的 GPU 加速实例

在阿里云部署 LLM 时，需要在单卡、多卡或分布式多节点等实例类型之间做取舍，兼顾性能需求与预算。除了模型参数量，还要考虑运行时资源需求，例如 KV Cache：

* **模型参数规模：** 参数越多，GPU 显存需求越大。正如前面章节所述，1.5B 参数模型约需 5.59 GB 显存（FP32 精度），而未压缩的 **DeepSeek-R1**（671B 参数）在 FP8 精度下至少需要 $\frac{671 \times 10^9}{2^{30}} \approx 625$ GB。
* **KV Cache 开销：** 在文本生成等场景中，KV Cache 用于存储注意力的键值对，其大小随着序列长度和批量大小增长。例如处理 2048 Token 的上下文就可能占据大量显存。
* **精度设置：** 不同精度（FP32、FP16、INT8、INT4）影响显存。量化技术可大幅减少占用。


以 **DeepSeek-R1（671B）** 为例。在 FP8 精度下，模型本体需约 625 GB 显存。若使用 MLA（多头潜在注意力）优化 KV Cache，假设每个 Token 每卡需 ~70 KB 缓存，8 卡系统处理 64K Token 请求需约 35 GB KV Cache。

总计显存约 660 GB，适配 `ecs.ebmgn8v.48xlarge` 实例（8 × 96 GB = 768 GB）。

最大并发量受 GPU 显存限制：

$$
\frac{req \times l \times (70 \times 1024)}{2^{30}} + \frac{625}{8} < 96 \text{ GB}
$$

其中 $req$ 为并发请求数，$l$ 为 Token 长度。

假设 $l = 64K$，可得 $req \approx 4$。由于显存紧张，系统同时只能服务少量用户。若要提升并发，可限制输入长度或升级到更大显存实例。

<br>

#### 选择合适的计费模式

选好实例后，需要根据业务需求确定计费方式：

|  | [包年包月](https://www.alibabacloud.com/help/en/ecs/subscription) | [按量付费](https://www.alibabacloud.com/help/en/ecs/pay-as-you-go-1) | [抢占式实例](https://www.alibabacloud.com/help/en/ecs/spot-instance) |
|----|----|----|----|
| **描述** | 预付费预留资源并享受较大折扣，适合长期稳定负载。 | 按需使用资源，无需预付，部署灵活快捷。<br>对于中长期使用，可结合 [节省计划](https://www.alibabacloud.com/help/en/ecs/savings-plans) 降低成本。 | 使用后付费，价格随供需变化，相较按量付费可节省最高 90%。 |
| **推荐场景** | 稳定、长期部署（如数月或数年） | 短期或用量波动的工作负载 | 能容忍短暂中断、以成本优先的非关键任务。<br>建议搭配重试机制或容错设计。<br>`在线推理使用抢占式实例：` [PAI-EAS Spot 最佳实践](https://www.alibabacloud.com/help/en/pai/use-cases/pai-eas-spot-best-practices) |

我们也建议使用 [预算管理](https://usercenter2-intl.console.alibabacloud.com/expense-manage/expense-budget/list) 设置成本告警，并定期查看 [成本分析](https://usercenter2-intl.console.alibabacloud.com/expense-manage/expense-analyze)，找出高成本组件（如 GPU/ECS 实例）。通过测试不同配置权衡成本与性能，在高负载时优先保障核心功能（如检索），同时关闭非关键服务（如复杂生成任务），即可优化整体支出。

## 接下来做什么？

## 小测验自查！

<details>
<summary style="cursor: pointer; padding: 12px; border: 1px solid #dee2e6; border-radius: 6px;">
<b>1. 使用小型蒸馏模型相较于大型通用模型的主要好处是什么？</b>  

<ul>
    <li>A) 在所有任务上精度更高</li>
    <li>B) 推理更快、成本更低、领域表现更好</li>
    <li>C) 更大的上下文窗口</li>
    <li>D) 无需微调</li>
</ul>

**查看答案 →**
</summary>

<div style="margin-top: 10px; padding: 15px; border: 1px solid #dee2e6; border-radius: 0 0 6px 6px;">

✅ **正确答案：** B) 推理更快、成本更低、领域表现更好  
📝 **解析：**  
小模型，尤其是针对特定领域蒸馏过的模型（如 Qwen-Math），在针对性任务上往往优于大模型，同时消耗更少资源，因此响应更快、成本更低。

</div>
</details>

<details>
<summary style="cursor: pointer; padding: 12px; border: 1px solid #dee2e6; border-radius: 6px; margin-top: 10px;">
<b>2. 哪项技术能通过复用已计算的上下文来降低推理成本？</b>  

<ul>
    <li>A) 模型剪枝</li>
    <li>B) 上下文缓存</li>
    <li>C) 提示词压缩</li>
    <li>D) 数据并行</li>
</ul>

**查看答案 →**
</summary>

<div style="margin-top: 10px; padding: 15px; border: 1px solid #dee2e6; border-radius: 0 0 6px 6px;">

✅ **正确答案：** B) 上下文缓存  
📝 **解析：**  
上下文缓存会保存先前交互的向量或键值状态，让模型无需重新处理共享上下文。在 Model Studio 等平台上，这可以把输入 Token 成本降低多达 60%。

</div>
</details>

<details>
<summary style="cursor: pointer; padding: 12px; border: 1px solid #dee2e6; border-radius: 6px; margin-top: 10px;">
<b>3. LLM 性能优化的“黄金法则”是什么？</b>  

<ul>
    <li>A) 永远使用最大的模型</li>
    <li>B) 为了完整性最大化输出 Token</li>
    <li>C) 最快的计算就是无需执行的计算</li>
    <li>D) 无论场景如何都启用流式输出</li>
</ul>

**查看答案 →**
</summary>

<div style="margin-top: 10px; padding: 15px; border: 1px solid #dee2e6; border-radius: 0 0 6px 6px;">

✅ **正确答案：** C) 最快的计算就是无需执行的计算  
📝 **解析：**  
这一原则支撑了缓存、硬编码、预计算与精简输入等策略——它们通过避免不必要的 LLM 调用来提升速度、降低成本。

</div>
</details>

---

## 关键要点

* **什么样的 LLM 应用才算生产就绪**
    * **从需求开始**——先定义功能性（做什么）与非功能性（做得如何）需求。
    * **速度至关重要**——用户期待亚秒级响应。制定 SLO（如聊天机器人 TTFT < 500ms）。
    * **成本增长很快**——每个 Token、每次推理都会累积，越早优化越好。
    * **稳定性不可妥协**——系统必须具备优雅降级能力（兜底、断路器、监控）。

<br>

* **性能优化是全栈工作**
    * **更快的处理：** 使用小型、量化或蒸馏模型。
    * **更少的请求：** 缓存上下文、请求去重、预计算答案。
    * **更少的 Token：** 精简输入、限制输出长度、使用摘要预处理。
    * **并行化：** 运用数据/模型/流水线并行来扩展吞吐量。
    * **能不用 LLM 就不用：** 对确定性响应采用硬编码或预计算。

<br>

* **用户体验不限于延迟**
    * **流式输出** 带来即时反馈体验。
    * **进度提示** 让用户在多步骤任务中保持安心。
    * **优雅的错误处理** 维持信任——绝不暴露系统堆栈。
    * **反馈按钮**（👍/👎）为持续改进提供数据。
    * **智能跟进** 让对话自然而顺畅地延展。

<br>

* **可靠性需要工程规范**
    * **自动扩缩容** 在不超额预配的情况下应对流量高峰。
    * **限流** 防止滥用，保障公平体验。
    * **监控** 持续关注延迟、错误、Token 与缓存命中率。
    * **灾备方案** 包括兜底模型、断路器与数据备份。
    * **把 LLM 当微服务管理**——应用 SRE 实践（SLA、SLO、复盘）。

<br>

* **成本优化是一项策略工程**
    * **小模型往往更优**——尤其是蒸馏或微调后。
    * **缓存能降低成本**——复用上下文比重算便宜。
    * **批处理省钱**——离峰批量任务成本可降 50%。
    * **合理选择计费方式**——稳定负载选包年包月，容错任务选抢占式。
    * **量化一切**——利用成本分析工具找出开销热点。

<br>

* **最好的系统往往是不调用 LLM 的系统**
    * **硬编码** 几乎零延迟、零成本，非常适合常见回复。
    * **预计算** 以一次性投入换取运行效率，适合 FAQ 等场景。
    * **结构化 UI**（表格、图表）比长段文本更快更清晰。
    * **基于规则的过滤** 可在到达模型之前就完成分流或解决。